# How to GeoCode
The purpose of this notebook is to illustrate how to use `ZGeo`, a module that converts address to Census Geographic Identifiers (GEOIDs).

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser, dirname
import pandas as pd
import sys
import os
import re
import warnings

In [3]:
warnings.filterwarnings(action='ignore')
home = expanduser('~')

src_path = '{}/zrp'.format(home)
sys.path.append(src_path)

In [4]:
from zrp.prepare.prepare import ProcessStrings, ZRP_Prepare, ZGeo
from zrp.prepare.utils import load_file

## Load sample data for prediction
Load processed list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [5]:
nj_mayors = load_file("../2022-nj-mayors-sample.csv")
nj_mayors.shape

(462, 9)

In [6]:
nj_mayors

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,ZEST_KEY
0,Gabe,NaN,Plumer,782,Frenchtown Road,Milford,NJ,08848,2
1,Ari,NaN,Bernstein,500,West Crescent Avenue,Allendale,NJ,07401,4
2,David,J.,Mclaughlin,125,Corlies Avenue,Allenhurst,NJ,07711-1049,5
3,Thomas,C.,Fritts,8,North Main Street,Allentown,NJ,08501-1607,6
4,P.,NaN,McCkelvey,49,South Greenwich Street,Alloway,NJ,08001-0425,7
...,...,...,...,...,...,...,...,...,...
457,William,NaN,Degroff,3943,Route,Chatsworth,NJ,08019,558
458,Joseph,NaN,Chukwueke,200,Cooper Avenue,Woodlynne,NJ,08107-2108,559
459,Paul,NaN,Sarlo,85,Humboldt Street,Wood-Ridge,NJ,07075-2344,560
460,Craig,NaN,Frederick,120,Village Green Drive,Woolwich Township,NJ,08085-3180,562


### Process the data
To quickly process the data we will use `ProcessStrings`.

In [7]:
str_processer = ProcessStrings()
str_processer.fit(nj_mayors)
nj_mayors = str_processer.transform(nj_mayors)

   [Start] Validating input data
     Number of observations: 462
     Is key unique: True
       (Warning!!) middle_name is 68.3982683982684% missing
Directory already exists
   [Completed] Validating input data

   Formatting P1
   Formatting P2
   reduce whitespace


#### Geocode  
To map addresses to GEOIDs we will use `ZGeo` 

Input data into the prediction/modeling pipeline is tabluar data with the following columns: first name, middle name, last name, house number, street address (street name), city, state, zip code, and zest key. The `ZEST_KEY` should be specified to establish correspondence between inputs and outputs; it's effectively used as an index for the data table. Geocoding does not require first name, middle name, and last name but it is best practice to include these columns if the intention is to return race & ethnicity proxies.

`ZGeo` is used to map addresses to block group, census tract, and other Census geographic identifiers. When called, the `.transform()` function's processing steps can include processing input data and geocoding the data. 

In [8]:
%%time
geocode = ZGeo()
geocode.fit()

CPU times: user 0 ns, sys: 43 µs, total: 43 µs
Wall time: 49.8 µs


Notes about the `.transform()` parameters:
- `geo = 34` indicates we want to geocode addresses from NJ
    - The state FIPs for NJ is 34. The user is required to input the numeric state FIPs if using ZGeo alone. Please refer to the inverse_state_mapping.json for assitance with mapping state abbreviation to state FIPs.
- The output data may be larger in size than the input data since `replicate` is set to True.
- No data is out because `save_table` is set to False. If True then the geocoded data will be saved to a file by state fips


In [9]:
zrp_output = geocode.transform(nj_mayors, geo='34', processed=False, replicate=True, save_table=False)


   Data is loaded
   [Start] Processing geo data
The key is already set
      ...formatting


  0%|          | 0/462 [00:00<?, ?it/s][Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 15 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 170 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 420 tasks      | elapsed:    0.0s
100%|██████████| 462/462 [00:00<00:00, 15202.60it/s]

      ...address cleaning
      ...replicating address
         ...Base
         ...Number processing...
         House number dataframe expansion is complete! (n=462)
         ...Base
         ...Map street suffixes...



[Parallel(n_jobs=-1)]: Done 462 out of 462 | elapsed:    0.0s finished


         ...Mapped & split by street suffixes...
         ...Number processing...

         Address dataframe expansion is complete! (n=900)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data
      ...merge user input & lookup table
      ...mapping
   [Completed] Validating input geo data
   [Completed] Mapping geo data


### Inspect the output


In [10]:
zrp_output.head()

,first_name,middle_name,last_name,house_number,street_address,city,state,zip_code,house_number_LEFT,house_number_RIGHT,...,big,ZIP_Match_1,ZIP_Match_2,FROMHN_numeric,TOHN_numeric,house_numer_numeric,GEOID_CT,GEOID_BG,GEOID_ZIP,GEOID
ZEST_KEY,,,,,,,,,,,,,,,,,,,,,
10,JOHN,M,MORGAN,137,MAIN STREET,ANDOVER,NJ,07821,None,137,...,None,None,None,None,None,None,34037373500,None,07821,None
100,MICHAEL,L,TEMPLETON,770,COOPERTOWN ROAD,DELANCO,NJ,08075,None,770,...,None,None,None,None,None,None,34005700800,None,08075,None
102,GARY,None,CATRAMBONE,900,CHESTER AVENUE,DELRAN,NJ,08075,None,900,...,None,None,None,None,None,None,None,None,08075,None
103,MELINDA,None,IANNUZZI,118,SERPENTINE ROAD,DEMAREST,NJ,07627,None,118,...,None,None,None,None,None,None,None,None,07627,None
104,ZETH,A,MATALUCCI,571,PETERSBURG ROAD,DENNISVILLE,NJ,08214,None,571,...,None,None,None,None,None,None,None,None,08214,None
